# Import libraries and data

In [1]:
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Get annotated tweets
tweets = pd.read_csv('../data/local/labelled_tweets_clean_all_mturk_agg.csv').reset_index()
tweets.head()

,index,text,neutral,negative,positive,total,agree_mturk
0,0,"""#Shingrix Vaccine Against Shingles Remains In...",3,0,0,3,neutral
1,1,"""...French scientist Louis Pasteur coined the ...",3,0,0,3,neutral
2,2,"""...due to potentially unexplained illnesses.""...",2,1,7,10,positive
3,3,"""...on average, it takes 10.7 years to develop...",3,0,0,3,neutral
4,4,"""..only 29 percent of the teens its members in...",3,0,0,3,neutral


In [3]:
# Get annotated tweets
tweets_1000 = pd.read_csv('../data/local/labelled_tweets_clean_all_mturk_agg_sample1000.csv').reset_index()
tweets_1000.head()

,index,text,neutral,negative,positive,total,agree_mturk,id_tweets
0,0,@user Deploy the J&J one dose vaccine to all c...,3,0,0,3,neutral,515
1,1,There's a petition on <url> that's requesting ...,4,0,0,4,neutral,3833
2,2,"This week: #outbreaks in #China, #Nigeria & #S...",3,0,0,3,neutral,3908
3,3,#ChristmasPuns How does Darth Vader like his C...,3,0,0,3,neutral,103
4,4,Colombia to participate in phase three vaccine...,2,1,8,11,positive,1282


In [3]:
# Get annotated tweets
tweets_1000_prop_missing = pd.read_csv('../data/local/labelled_tweets_clean_all_mturk_agg_sample1000_proportional_missing.csv').reset_index()
tweets_1000_prop_missing.head()

,index,text,neutral,negative,positive,total,agree_mturk,id_tweets
0,0,FYI There is NO WAY to implant a chip in your ...,2,4,4,10,negative,1654
1,1,The #covid lies are real- #NoVaccineForMe I wi...,2,5,3,10,negative,3600
2,2,More than a third of South Africans would not ...,4,5,1,10,negative,2716
3,3,i dont usually get into politics but i dont un...,0,5,4,9,negative,4599
4,4,i would drink the presidents fluids for the co...,0,9,1,10,negative,4607


# Using sample to test prompt

In [4]:
# Prompt for GPT
prompt_raw_1 = "Give me the sentiment regarding vaccination expressed by the user of the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [95]:
prompt_raw_0 = prompt_raw
prompt_raw_0

'Give me the sentiment regarding vaccination in tweet text delimited by triple backticks. Use one of the following words: neutral, negative, positive. ```{TWEET_TEXT_HERE}```'

In [82]:
prompt_raw_1.write('../data/local/gpt_prompt_1.csv')


AttributeError: 'str' object has no attribute 'write'

In [5]:
# Prompt for GPT
prompt_raw_1 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
```{TWEET_TEXT_HERE}```"

In [6]:
prompt_raw_3 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral.\
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [7]:
prompt_raw_4 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [8]:
prompt_raw_5 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
* The user is ironic on people being negative towards vaccine (e.g., I am not getting a vaccine because they are microchipping you. Posted on social media , on a phone..that has all your info, your location...and the info is being shared and sold ...Ummmm ok) \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [9]:
prompt_raw_6 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
\
```{TWEET_TEXT_HERE}```"

In [10]:
prompt_raw_7 = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. Include an explanation for the selection of the sentiment. \
\
News, factual or objective tweets as well as generally ambiguous examples should be classified as neutral except if:\
* User agrees with a positive statement on vaccines or vaccinations which should be classified as positive \
* User disagrees with a positive statement on vaccines or vaccinations which should be classified as negative \
* User is angry or not happy when vaccines are not available which should be classified as positive \
* User is angry or not happy with a negative statement on vaccines or vaccinations which should be classified as positive \
* User is happy with a negative statement on vaccines or vaccinations which should be classified as negative \
* User is enquiring for availability or increased availability of a vaccine which should be classified as positive \
\
Classify as positive if:\
* The user expresses hope or anticipation towards a vaccine.\
* The user expresses desire for having more vaccine doses. \
\
Examples of positive tweets: \
* Some news of hope. Hopefully this vaccine becomes success and India becomes leader again. \
* This has been a horrendous year for so many people who have lost loved ones. It would be wonderful if the #vaccine was effective and could be administered world-wide to prevent further loss. My fingers and toes are crossed. \
* A healthy recovery of the world from COVID-19 is only possible when no one is left behind making it necessary that the vaccine is accessible to all without any hurdle. #HealthForAll #Equality #Equity #publichealth #awarness. \
* IF I SEE ONE MORE FUCKING POLITICIAN GET THE VACCINE BEFORE FRONT LINE WORKERS I'M GONNA LOSE MY SHIT!! \
* When will we learn to respect our scientific community ? Pathetic that ppl want to play politics on the vaccine \
* @user @user @user etc if U have so much problem so plz don't get them vaccinated, we know, U have enough money for treatment. U guys fall mild sick, immediately rush to medanta hospital.\
* $MRNA recieved approval across the pond in the U.K. demand increasing globally for a vaccine as cases rise. #stocks #finance #markets #investing #money #wealth #economy #OptionsTrading \
* On the other hand, they're more likely to receive much more effective and safe vaccines, so it's not quite as bad as you might think \
* Why are not more COVID-19 vaccine doses available immediately? <url> \
* America have fewer COVID-19 vaccines due to #MassMurderer #realDonaldTrumpâ€™s FAILED financial interests <url> \
\
Examples of neutral tweets: \
* AstraZeneca Pauses Vaccine Trial After Test Subject Gets Ill <url> \
* The federal government outlined a sweeping plan Wednesday to make vaccines for COVID-19 available for free to all Americans. <url> \
\
Examples of negative tweets: \
* There's a reason these vaccines can take up to 20 years to reach market. Say no to the vaccine. \
* Vaccines may not be good for kids like the Dengue Fever vaccine. You’re playing with fire. <url> \
* Dr: Are you experiencing any side effects from the vaccine? Me: * sleeps while standing there* huh? \
* There is a virus which has killed so many people for years. This virus is called hunger and the vaccine for hunger is food. However no one talks about it, why? Because this virus doesn't kill rich people.\
* @user : There are over 100 different coronavirus vaccine candidates in the works. These candidates take a variety of approaches to protecting the body against COVID-19: <url> #CoronaVirusUpdates #CoronavirusVaccine #StayHome <url> \
* <It would go away without the vaccine> #HerdMentality this man knows nothing about viruses. \
* Anti-vaxxer Novak Djokovic. #COVID19 #tennis The current world number one is the fourth player to test positive from the event that did not even pay lip service to social distancing. <url> via @user \
\
```{TWEET_TEXT_HERE}```"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [13]:
len(tweets_1000_prop_missing.index)

785

In [12]:
tweets_1000_prop_missing.iloc[1,1]

'The #covid lies are real- #NoVaccineForMe I will trust my immune system as I have all my life'

In [20]:
id_gpt = []
sent_gpt = []

In [21]:
for i in range(0, 785):
    prompt_i = prompt_raw_1.replace('TWEET_TEXT_HERE', tweets_1000_prop_missing.iloc[i,1])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt.append(tweets_1000_prop_missing.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

RateLimitError: The server had an error while processing your request. Sorry about that!

In [22]:
df_gpt_total = pd.DataFrame(list(zip(id_gpt, sent_gpt)),
                            columns = ['id_gpt', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_gpt         5 non-null      int64 
 1   sentiment_gpt  5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [18]:
df_gpt_total['prompt'] = 1

In [23]:
df_gpt_total.tail()

,id_gpt,sentiment_gpt
0,0,\n\nNegative. The user expresses disbelief and...
1,1,\n\nNegative. The user expresses dissatisfacti...
2,2,\n\nNegative. The implication of this tweet is...
3,3,\n\nNegative. The user expresses their confusi...
4,4,\n\nNegative. The tweet expresses strong antip...


In [19]:
df_gpt_total_all.head()

,id_gpt,sentiment_gpt,prompt
0,1642,\n\nNeutral. This tweet does not express any s...,5
1,1643,Neutral. This is a factual and objective stat...,5
2,1644,Neutral. The user does not express any opinio...,5
3,1645,Neutral. The tweet is factual and does not ex...,5
4,1646,\n\nNeutral. This tweet does not express a sen...,5


In [12]:
#df_gpt_total_all = df_gpt_total

In [20]:
df_gpt_total_all = df_gpt_total_all.append(df_gpt_total)

C:\Users\lespinosa\AppData\Local\Temp\ipykernel_40216\964839105.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gpt_total_all = df_gpt_total_all.append(df_gpt_total)


In [21]:
len(df_gpt_total_all)

3144

In [22]:
df_gpt_total_all.tail()

,id_gpt,sentiment_gpt,prompt
1847,4781,Positive. The user expresses a desire for mor...,5
1848,4782,\n\nNeutral. The tweet does not express a sent...,5
1849,4783,\n\nPositive. The user expresses a sentiment o...,5
1850,4784,Neutral. The tweet is a factual statement tha...,5
1851,4785,Neutral. The tweet expresses admiration for t...,5


In [23]:
df_gpt_total_all.to_csv('../data/local/gpt_sentiment_mturk_prompt5all0_1.csv')

# Using three categories for vaccine sentiment

In [10]:
# Prompt for GPT
prompt_raw = "Give me the sentiment regarding vaccination expressed by the user in the tweet text delimited by triple backticks. \
Use one of the following words: neutral, negative, positive. \
```{TWEET_TEXT_HERE}```"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [11]:
len(tweets.index)

4786

In [12]:
test = tweets.iloc[:101, :]
test.info

<bound method DataFrame.info of      index                                               text  neutral  \
0        0  "#Shingrix Vaccine Against Shingles Remains In...        3   
1        1  "...French scientist Louis Pasteur coined the ...        3   
2        2  "...due to potentially unexplained illnesses."...        2   
3        3  "...on average, it takes 10.7 years to develop...        3   
4        4  "..only 29 percent of the teens its members in...        3   
..     ...                                                ...      ...   
96      96  #COVID19 vaccine could be delivered in mailabl...        4   
97      97  #COVID19Vaccine in 1 month? CSIR pins hope on ...        4   
98      98  #COVID19dz Ignorance at the utmost level as co...        1   
99      99  #CRO Vaccine Market in Austria to 2021 Market ...        3   
100    100  #California has just implemented one of the #s...        3   

     negative  positive  total  
0           0         0      3  
1           0

In [7]:
tweets1 = tweets.iloc[0:306,:]
tweets2 = tweets.iloc[306:601,:]
tweets3 = tweets.iloc[601:901, :]
tweets4 = tweets.iloc[901:1201, :]


In [14]:
test.iloc[1,1]

'"...French scientist Louis Pasteur coined the term “vaccination” in honour of Jenner’s work (vacca being the Latin for cow)" - TIL! 🐮💉 <url>'

In [68]:
id_gpt = []
sent_gpt = []

In [69]:
for i in range(0,4785):
    prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[i,1])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.8,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt.append(tweets.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

In [38]:
run = np.array([1])

In [70]:
df_gpt_total = pd.DataFrame(list(zip(id_gpt, sent_gpt)),
                            columns = ['id_gpt', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4785 entries, 0 to 4784
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_gpt         4785 non-null   int64 
 1   sentiment_gpt  4785 non-null   object
dtypes: int64(1), object(1)
memory usage: 74.9+ KB


In [71]:
df_gpt_total['run'] = 3

In [72]:
df_gpt_total.tail()

,id_gpt,sentiment_gpt,run
4780,4780,\n\nNegative,3
4781,4781,\n\npositive,3
4782,4782,\n\nneutral,3
4783,4783,\n\npositive,3
4784,4784,\n\nNeutral,3


In [59]:
#df_gpt_total_all = df_gpt_total.append(df_gpt_total2)

C:\Users\lespinosa\AppData\Local\Temp\ipykernel_23560\3678373193.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gpt_total_all = df_gpt_total.append(df_gpt_total2)


In [73]:
df_gpt_total.to_excel('../data/local/gpt_sentiment_mturk_v4.xlsx')


# Using two categories for vaccine sentiment

In [6]:
tweets_pos_neg = tweets[tweets['label_tag'].isin(['positive', 'negative'])]
tweets_pos_neg.head()

,index,id,text,label_tag
18,18,1235574994924490752,@user @user I saw #VaxxedII yesterday and I ha...,negative
22,22,1236451808412909568,Dear anti-vaxxers. If you are legitimately wor...,positive
26,26,1237492335430250496,@user @user thank you for the vaccine LOL #Wel...,positive
28,28,1237894709306986496,Remember polio? Killed a lot of people. Vaccin...,positive
38,38,1239077015250030592,You be ready to pay what this Coronavirus vacc...,negative


In [39]:
tweets_new_pos_neg = tweets_new[tweets_new['label_tag'].isin(['positive', 'negative'])]
tweets_new_pos_neg.head()

,index,id,text,label_tag
1,1,1008751928568840192,"Great news to see. Hopefully, folks who disput...",positive
2,2,1008751948957351936,The global community must invest in further re...,positive
3,3,1008752099222523904,HPV vaccine has almost wiped out infections in...,positive
4,4,1010156916553736192,So I can catch this shit too and need treatmen...,negative
5,5,1010157450618601472,Could a #vaccine for #type-1 diabetes be on th...,positive


In [42]:
prompt_raw_pos_neg = "Please give me the sentiment regarding vaccination in the following tweet: \
"TWEET_TEXT_HERE\"\nUse one of the following words: negative, positive"

In [40]:
len(tweets_new_pos_neg.index)

128

In [9]:
test = tweets_pos_neg.iloc[0:2, :]
test

,index,id,text,label_tag
18,18,1235574994924490752,@user @user I saw #VaxxedII yesterday and I ha...,negative
22,22,1236451808412909568,Dear anti-vaxxers. If you are legitimately wor...,positive


In [7]:
tweets_pos_neg1 = tweets_pos_neg.iloc[0:306,:]
tweets_pos_neg2 = tweets_pos_neg.iloc[306:601,:]
tweets_pos_neg3 = tweets_pos_neg.iloc[601:901, :]
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]


In [103]:
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]

In [43]:
id_gpt_two = []
sent_gpt_two = []

In [44]:
for i in range(0,128):
    prompt_two_i = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_new_pos_neg.iloc[i,2])
    response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_i,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt_two.append(tweets_new_pos_neg.iloc[i,0])
    sent_gpt_two.append(response_two.choices[0].text)

In [17]:
prompt_two_1339 = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_pos_neg.iloc[1339,2])
response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_1339,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
id_gpt_two.append(tweets_pos_neg.iloc[1339,0])
sent_gpt_two.append(response_two.choices[0].text)

In [46]:
df_two_gpt_total = pd.DataFrame(list(zip(id_gpt_two, sent_gpt_two)),
                            columns = ['id_gpt_two', 'sentiment_gpt_two'])
df_two_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_gpt_two         128 non-null    int64 
 1   sentiment_gpt_two  128 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


In [47]:
df_two_gpt_total.tail()

,id_gpt_two,sentiment_gpt_two
123,179,", or neutral\n\nPositive"
124,180,", or neutral\n\nPositive"
125,181,", or neutral\n\nNegative"
126,182,", or neutral\n\nPositive"
127,184,\n\nNegative


In [108]:
id_gpt13 = id_gpt
sent_gpt13 = sent_gpt
df_gpt_total13 = df_gpt_total

In [25]:
#df_gpt_total1.to_csv('../data/local/gpt_sentiment1.csv')
df_gpt_total1['sentiment_gpt2'] = df_gpt_total1['sentiment_gpt'].to_string()
print(type(df_gpt_total1['sentiment_gpt2']))

<class 'pandas.core.series.Series'>


In [30]:
#sent_gpt1.to_csv('../data/local/gpt_sentiment1_only.csv')

In [48]:
df_two_gpt_total.to_excel('../data/local/gpt_sentiment_new_pos_neg.xlsx')
